In [1]:
import functools
from typing import Callable, Union

import clingo
import clingo.ast
from clingo.ast import ProgramBuilder
from clingox.backend import SymbolicBackend

In [2]:
yale = """

% beliefs {lazy, pacifist}
% actions {load, shoot}
% observations {alive, dead, loaded, unloaded}.

load :- not lazy.
lazy :- not load.

shoot :- not pacifist.
pacifist :- not shoot.

loaded :- load.
unloaded :- not loaded.
dead :- shoot, loaded.
alive :- not shoot.
alive :- unloaded.

"""

In [3]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0

In [4]:
ctl.add("base", (), yale)

In [5]:
ctl.ground([("base", [])])

In [6]:
with ctl.solve(yield_=True) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = model.symbols(shown=True)
        print("Answer {}:".format(model.number), "{",
              ' '.join(map(str, sorted(symbols))), "}")
        models.append(symbols)
    solve_result = solve_handle.get()
    print(solve_result)

Answer 1: { dead load loaded shoot }
Answer 2: { alive load loaded pacifist }
Answer 3: { alive lazy shoot unloaded }
Answer 4: { alive lazy pacifist unloaded }
SAT


In [7]:
nodes = []
clingo.ast.parse_string(yale, lambda stm: nodes.append(stm))
print('\n'.join(map(str, nodes)))

#program base.
load :- not lazy.
lazy :- not load.
shoot :- not pacifist.
pacifist :- not shoot.
loaded :- load.
unloaded :- not loaded.
dead :- shoot; loaded.
alive :- not shoot.
alive :- unloaded.


In [8]:
model = models[0]
print('. '.join(map(str, model)), '.', sep='')

load. shoot. loaded. dead.


In [9]:
def ast_match(node: clingo.ast.AST, symbol: clingo.Symbol):
    print(node, '=?=', symbol)
    if node.ast_type is clingo.ast.ASTType.SymbolicTerm:
        return node.symbol == symbol
    elif node.ast_type is clingo.ast.ASTType.Function:
        return ast_function_match(node, symbol)
    else:
        return False


def ast_function_match(function: clingo.ast.AST, symbol: clingo.Symbol):
    if function.ast_type is not clingo.ast.ASTType.Function or symbol.type is not clingo.SymbolType.Function:
        print(function.ast_type, '=x=', clingo.SymbolType)
        return False
    else:
        matches = True
        if function.name != symbol.name:
            print(function.name, '=/=', symbol.name)
            matches = False
        if len(function.arguments) != len(symbol.arguments):
            print(len(function.arguments), '=/=', len(symbol.arguments))
            matches = False
        if matches:
            print(function, '==', symbol)
        return matches

In [10]:
# 1. Plan description
actions = [symbol for symbol in model if symbol.match('load', 0) or symbol.match('shoot', 0)]
actions

[Function('load', [], True), Function('shoot', [], True)]

In [11]:
# 2. Action justification: Why action X?
target_belief = actions[-1]
target_belief

Function('shoot', [], True)

In [12]:
support_axioms = [node for node in nodes if
                  node.ast_type is clingo.ast.ASTType.Rule and ast_match(node.head.atom.symbol, target_belief)]

load =?= shoot
load =/= shoot
lazy =?= shoot
lazy =/= shoot
shoot =?= shoot
shoot == shoot
pacifist =?= shoot
pacifist =/= shoot
loaded =?= shoot
loaded =/= shoot
unloaded =?= shoot
unloaded =/= shoot
dead =?= shoot
dead =/= shoot
alive =?= shoot
alive =/= shoot
alive =?= shoot
alive =/= shoot


In [13]:
print('\n'.join(map(str, support_axioms)))

shoot :- not pacifist.


In [14]:
pos = clingo.ast.Position('<string>', 1, 1)
loc = clingo.ast.Location(pos, pos)

In [15]:
ctl = clingo.Control()

In [16]:

with SymbolicBackend(ctl.backend()) as sb:
    for fact in model:
        sb.add_rule(head=[fact])

In [17]:

rules = []
for support_axiom in support_axioms:
    head = support_axiom.head.atom.symbol
    head_name = clingo.ast.SymbolicTerm(loc, clingo.Function(head.name))
    head_parameters = head.arguments
    body = support_axiom.body
    fires_head = clingo.ast.Literal(loc, sign=0, atom=clingo.ast.SymbolicAtom(
        clingo.ast.Function(loc, '__fires', (head_name,clingo.ast.Function(loc, '', head_parameters, False)), False)))
    fires_rule = clingo.ast.Rule(loc, fires_head, body)
    rules.append(fires_rule)
    for literal in body:
        symbol = literal.atom.symbol
        name = symbol.name
        args = symbol.arguments
        holds_head = clingo.ast.Literal(loc, sign=0, atom=clingo.ast.SymbolicAtom(
            clingo.ast.Function(loc, '__holds', (
                clingo.ast.SymbolicTerm(loc, clingo.Number(literal.sign)),
                clingo.ast.SymbolicTerm(loc, clingo.Function(name)),
                clingo.ast.Function(loc, '', args, False)
            ), False)
        ))
        holds_rule = clingo.ast.Rule(loc, holds_head, (fires_head,))
        rules.append(holds_rule)
print('\n'.join(map(str, rules)))

__fires(shoot,()) :- not pacifist.
__holds(1,pacifist,()) :- __fires(shoot,()).


In [18]:
with ProgramBuilder(ctl) as pb:
    for rule in rules:
        pb.add(rule)

In [19]:
ctl.add('base', (), '#show __fires/2. #show __holds/3.')

In [20]:
ctl.ground([('base', [])])
with ctl.solve(yield_=True) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = model.symbols(shown=True)
        print("Answer {}:".format(model.number), "{",
              ' '.join(map(str, sorted(symbols))), "}")
        models.append(symbols)
    solve_result = solve_handle.get()
    print(solve_result)


Answer 1: { __fires(shoot,()) __holds(1,pacifist,()) }
SAT


<string>:10:14-22: info: atom does not occur in any rule head:
  pacifist



In [21]:
yale_complex = """

time(0..2).

observe(object(agent, alice), value(gun, unloaded), 0).
observe(object(rooster, 0), value(alive, true), 0).
occurs(object(agent, alice), value(load, gun), 0).
occurs(object(agent, alice), value(shoot, gun), 1).

observe(object(rooster, 0), value(alive, false), T+1) :-
  time(T+1),
  observe(object(rooster, 0), value(alive, false), T).

observe(object(rooster, 0), value(alive, false), T+1) :-
  time(T+1),
  occurs(object(agent, alice), action(shoot, gun), T),
  observe(object(rooster, 0), value(alive, true), T),
  observe(object(agent, alice), value(gun, loaded), T).

observe(object(rooster, 0), value(alive, true), T+1) :-
  time(T+1),
  not occurs(object(agent, alice), action(shoot, gun), T),
  observe(object(rooster, 0), value(alive, true), T).

observe(object(rooster, 0), value(alive, true), T+1) :-
  time(T+1),
  observe(object(rooster, 0), value(alive, true), T),
  not observe(object(agent, alice), value(gun, loaded), T).

observe(object(agent, alice), value(gun, unloaded), T+1) :-
  time(T+1),
  occurs(object(agent, alice), action(shoot, gun), T),
  observe(object(agent, alice), value(gun, loaded), T).

observe(object(agent, alice), value(gun, loaded), T+1) :-
  time(T+1),
  not occurs(object(agent, alice), action(shoot, gun), T),
  observe(object(agent, alice), value(gun, loaded), T).


observe(object(agent, alice), value(gun, loaded), T+1) :-
  time(T+1),
  occurs(object(agent, alice), action(load, gun), T),
  observe(object(agent, alice), value(gun, unloaded), T).

observe(object(agent, alice), value(gun, unloaded), T+1) :-
  time(T+1),
  not occurs(object(agent, alice), action(load, gun), T),
  observe(object(agent, alice), value(gun, unloaded), T).

"""

In [22]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0

In [23]:
ctl.add("base", (), yale_complex)

In [24]:
ctl.ground([("base", [])])

In [25]:
def temporal(symbol: clingo.Symbol, default: Union[int, Callable[[clingo.Symbol], int], None] = None):
    if symbol.match('occurs', 3) or symbol.match('observe', 3):
        return symbol.arguments[-1].number
    if callable(default):
        return default(symbol)
    return default

In [26]:
with ctl.solve(yield_=True) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = model.symbols(shown=True)
        print("Answer {}:".format(model.number), "{",
              ' '.join(map(str, sorted(sorted(symbols), key=functools.partial(temporal, default=-1)))), "}")
        models.append(symbols)
    solve_result = solve_handle.get()
    print(solve_result)

Answer 1: { time(0) time(1) time(2) observe(object(agent,alice),value(gun,unloaded),0) observe(object(rooster,0),value(alive,true),0) occurs(object(agent,alice),value(load,gun),0) observe(object(agent,alice),value(gun,unloaded),1) observe(object(rooster,0),value(alive,true),1) occurs(object(agent,alice),value(shoot,gun),1) observe(object(agent,alice),value(gun,unloaded),2) observe(object(rooster,0),value(alive,true),2) }
SAT
